In the standard form of LP constraints have $(\leq)$ with non-negative righthand side. If we include $(=)$ or $(\geq)$ constraints with or without negative righthand side, then primitive simplex algorithm doesn't work. 

In order to solve these "ill-behaved" linear programming problems we have to use **artificial variable** that play the role of slack in first iteration. Later these variable are disposed.

## M-Method

**Example 3.4-1**

Consider the following linear programming problem.
$$
\left.\begin{array}{rrrrrl} \max & 4x &+& y \\
    \text {s.t.:}  \\
    & 3x &+& y &=&    3 \text{ (Constraint 1)}\\
    & 4x &+& 3y &\geq& 6 \text{ (Constraint 2)} \\
    &  x &+& 2y &\leq& 4 \text{ (Constraint 3)} \\
    &&&  x, y   &\geq& 0 \text{ (Non-negativity)} \\                
\end{array}\right\}
$$

This can be converted into follwing problem:

$$
\left.\begin{array}{llrrrrrrl} \max & 4x &+& y &+& MR_1 &+& MR_2\\
    \text {s.t.:}  \\
    & 3x &+&  y & &     & &     &+& R_1 & &      &=&    3 \text{ (Constraint 1)}\\
    & 4x &+& 3y &-& s & &     & &     &+& R_2  &=& 6 \text{ (Constraint 2)} \\
    &  x &+& 2y & &     &+& t & &     & &      &=& 4 \text{ (Constraint 3)} \\
    &  x, && y, && s, &&t, &&R_1, && R_2          &\geq& 0 \text{ (Non-negativity)} \\                
\end{array}\right\}
$$

Now if we initialize the simplex algorithm with very high value of negative $M$, then final solution of $R_1$ and $R_2$ will be $0$. The starting basic solution will be 

| Basic |  x |  y |  s |  R1 |  R2 | t | Solution |
|-------|---:|---:|---:|----:|----:|--:|---------:|
| z     | -4 | -1 |  0 | 100 | 100 | 0 |        0 |
| R1    |  3 |  1 |  0 |   1 |   0 | 0 |        3 |
| R2    |  4 |  3 | -1 |   0 |   1 | 0 |        6 |
| t     |  1 |  2 |  0 |   0 |   0 | 1 |        4 |

The value z row is not correct, we have to modify it as follows

$$ \text{New  } z\text{-row } = \text{ Old } z\text{-row} - 100\times R_1\text{-row} - 100\times R_2\text{-row} $$

The new tableau will will be as follows:

| Basic 	|    x 	|    y 	|  s 	| R1 	| R2 	| t 	| Solution 	|
|-------	|-----:	|-----:	|---:	|---:	|---:	|--:	|---------:	|
| z     	| -696 	| -399 	|  100 	|  0 	|  0 	| 0 	|     -900 	|
| R1    	|    3 	|    1 	|  0 	|  1 	|  0 	| 0 	|        3 	|
| R2    	|    4 	|    3 	| -1 	|  0 	|  1 	| 0 	|        6 	|
| t     	|    1 	|    2 	|  0 	|  0 	|  0 	| 1 	|        4 	|


In [3]:
t = Tableau([-696, -399, 100]) # starting value optimal will not effect the calculation.
t.add_constraint([3,1, 0], 3)
t.add_constraint([4,3,-1], 6)
t.add_constraint([1,2, 0], 4)
t.is_fraction = True
t.solve()

Basic         x1      x2      x3      s1      s2      s3    Sol.
z           -696    -399     100       0       0       0       0
s1             3       1       0       1       0       0       3
s2             4       3      -1       0       1       0       6
s3             1       2       0       0       0       1       4


Entering Variable:  x1
Leaving Variable :  s1


Basic         x1      x2      x3      s1      s2      s3    Sol.
z              0    -167     100     232       0       0     696
x1             1     1/3       0     1/3       0       0       1
s2             0     5/3      -1    -4/3       1       0       2
s3             0     5/3       0    -1/3       0       1       3


Entering Variable:  x2
Leaving Variable :  s2


Basic         x1      x2      x3      s1      s2      s3    Sol.
z              0       0    -1/3   295/3   301/3       0  2689/3
x1             1       0     1/3     2/3    -1/3       0     2/3
x2             0       1    -2/3    -2/3     2/3       

ValueError: I/O operation on closed file

In [1]:
"""
Modified from: http://hubpages.com/technology/Simplex-Algorithm-in-Python
"""

from __future__ import division
from numpy import *

# Ref: http://stackoverflow.com/questions/23344185/how-to-convert-a-decimal-number-into-fraction
from fractions import Fraction


class Tableau:
 
    def __init__(self, obj):
        self.obj = [1] + obj
        self.rows = []
        self.cons = []
        self.no_variables = len(obj)
        self.no_constraints = 0
        self.is_fraction = False # set True to output in fraction
 
    def add_constraint(self, expression, value):
        self.rows.append([0] + expression)
        self.cons.append(value)
        self.no_constraints += 1
               
        self.basic_variables = ["s"+str(i+1) for i in range(self.no_constraints)]
 
    def _pivot_column(self):
        low = 0
        idx = 0
        for i in range(1, len(self.obj)-1):
            if self.obj[i] < low:
                low = self.obj[i]
                idx = i
        if idx == 0: return -1
        return idx
 
    def _pivot_row(self, col):
        rhs = [self.rows[i][-1] for i in range(len(self.rows))]
        lhs = [self.rows[i][col] for i in range(len(self.rows))]
        ratio = []
        for i in range(len(rhs)):
            if lhs[i] == 0:
                ratio.append(99999999 * abs(max(rhs)))
                continue
            ratio.append(rhs[i]/lhs[i])
        return argmin(ratio)
 
    def display(self):   
        if self.is_fraction:
            # Formatting the output in fraction
            # Ref: https://pyformat.info/
            fmt = '{:<8}'.format("Basic") \
                  + "".join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)])   \
                  + "".join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) \
                  + '{:>8}'.format("Sol.")

            fmt += "\n" 
            fmt += '{:<8}'.format("z") \
                   + "".join(["{:>8}".format(Fraction(item).limit_denominator(3)) for item in self.obj[1:]])

            for i, row in enumerate(self.rows):
                fmt += "\n" 
                fmt += '{:<8}'.format(self.basic_variables[i]) \
                       + "".join(["{:>8}".format(Fraction(item).limit_denominator(3)) for item in row[1:]])
            print fmt
            
        else:
            # Formatting the output in float with 2 decimal places
            fmt = '{:<8}'.format("Basic") \
                  + "".join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)])   \
                  + "".join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) \
                  + '{:>8}'.format("Sol.")

            fmt += "\n" 
            fmt += '{:<8}'.format("z") + "".join(["{:>8.2f}".format(item) for item in self.obj[1:]])

            for i, row in enumerate(self.rows):
                fmt += "\n" 
                fmt += '{:<8}'.format(self.basic_variables[i]) \
                       + "".join(["{:>8.2f}".format(item) for item in row[1:]])
            print fmt            
              
#       print '\n', matrix([self.obj] + self.rows)
 
    def _pivot(self, row, col):
        e = self.rows[row][col]
        self.rows[row] /= e
        for r in range(len(self.rows)):
            if r == row: continue
            self.rows[r] = self.rows[r] - self.rows[r][col]*self.rows[row]
        self.obj = self.obj - self.obj[col]*self.rows[row]
 
    def _check(self):
        if min(self.obj[1:-1]) >= 0: return 1
        return 0
         
    def solve(self):
        # build full tableau
        for i in range(len(self.rows)):
            self.obj += [0]
            ident = [0 for r in range(len(self.rows))]
            ident[i] = 1
            self.rows[i] += ident + [self.cons[i]]
            self.rows[i] = array(self.rows[i], dtype=float)
        self.obj = array(self.obj + [0], dtype=float)
 
        # solve
        self.display()
        while not self._check():
            c = self._pivot_column()
            r = self._pivot_row(c)
            self._pivot(r,c)
            # print '\npivot column: %s\npivot row: %s'%(c+1,r+2)
            print '\n'
            print 'Entering Variable: ', self.header_tableau[c]
            print 'Leaving Variable : ', self.basic_variables[r]
            print '\n'
            # Updating the basic variable
            for index, item in enumerate(self.basic_variables):
                if self.basic_variables[index] == self.basic_variables[r]:
                    self.basic_variables[index] = self.header_tableau[c]
                               
            self.display()

In [2]:
dir(Tableau)

['__doc__',
 '__init__',
 '__module__',
 '_check',
 '_pivot',
 '_pivot_column',
 '_pivot_row',
 'add_constraint',
 'display',
 'solve']